#  1 _API_ Requests: Data Acquisition
---

* [02 the next notebook](02.ipynb)
* [03 the one after](03.ipynb)

---

### Data Discussion

* [Fuel data from US Energy Information Admininstration](https://www.eia.gov/opendata/qb.php?category=240839&sdid=PET.EMM_EPM0_PTE_SCA_DPG.M)
    * weekly data: [fuel_m.csv](../data/processed/fuel_w.csv) - [source](https://www.eia.gov/opendata/qb.php?category=240839&sdid=PET.EMM_EPM0_PTE_SCA_DPG.W)
    * monthly data: [fuel_w.csv](../processed/fuel_m.csv) - [source](https://www.eia.gov/opendata/qb.php?category=240839&sdid=PET.EMM_EPM0_PTE_SCA_DPG.M)
    

* [BART](https://data.bart.gov/dataset/customer-ridership/resource/6e653520-58cf-45c5-b40c-d37c8957ec77) Publishes monthly rerpots, with daily ridership that month, using faregate counts for on and off boarding. 
    * [montly totals](https://data.bart.gov/group/ridership)
    * monthly by entrance and exit, saved 

* Car registration data from 
    * by county: []()
    * totals : []()

* will not be using: ~~[caltrain](caltrain.com)  Publishes yearly ridership counts, based on model validated with a yearly county daily in January.~~

In [1]:
##### BASIC IMPORTS 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import requests

In [2]:
# CUSTOM IMPORTS AND SETTINGS 
plt.style.use('gstyle.mplstyle')

pd.options.display.max_columns = 90                    
pd.options.display.max_rows = 100

## EIA DATA: Fuel Price History 

In [22]:
# EIA KEY 
try:
    KEY = os.environ['EIAAPI']
except KeyError:
    sys.exit('keys not found')

#### Function for EIA requests takes KEY and CATEGORY

In [28]:
# function for EIA data requests
def eia_req(KEY, CATEGORY):
    url = 'https://api.eia.gov/series/?api_key=' + KEY + '&series_id=' + CATEGORY
    
    #REQUEST 
    req = requests.get(url)
    req.status_code

    # getting data 
    request = req.json()['series'][0]['data']
    data = pd.DataFrame(request)

    return (data)

#### Fuel prices by month

In [38]:
CATEGORY = 'PET.EMM_EPM0_PTE_SCA_DPG.M'

# call function for pull 
fuel_m = eia_req(KEY, CATEGORY)
fuel_m.columns = ['date', 'price']
fuel_m.head()

,date,price
0,202204,5.692
1,202203,5.655
2,202202,4.660
3,202201,4.584
4,202112,4.597


In [39]:
# add new cols from old date colum
fuel_m['year'] = fuel_m['date'].str[:4]
fuel_m['month'] = fuel_m['date'].str[-2:]

# drop old date info 
fuel_m.drop(columns = 'date', inplace = True)
fuel_m.head()

,price,year,month
0,5.692,2022,04
1,5.655,2022,03
2,4.660,2022,02
3,4.584,2022,01
4,4.597,2021,12


In [40]:
# print to file
fuel_m.to_csv('../data/processed/fuel_m.csv')

#### Fuel Prices by Week

In [41]:
CATEGORY = 'PET.EMM_EPM0_PTE_SCA_DPG.W'

# call function for pull 
fuel_w = eia_req(KEY, CATEGORY)
fuel_w.columns = ['date', 'price']
fuel_w.head()

,date,price
0,20220502,5.629
1,20220425,5.609
2,20220418,5.641
3,20220411,5.715
4,20220404,5.804


In [53]:
# add new cols from old date colum
fuel_w['year'] = fuel_w['date'].str[:4]
fuel_w['month'] = fuel_w['date'].str[4:6]
fuel_w['day'] = fuel_w['date'].str[-2:]

# drop old date info 
fuel_w.drop(columns = 'date', inplace = True)
fuel_w.head()

,price,year,month,day
0,5.629,2022,05,02
1,5.609,2022,04,25
2,5.641,2022,04,18
3,5.715,2022,04,11
4,5.804,2022,04,04


In [54]:
# print to file
fuel_w.to_csv('../data/processed/fuel_w.csv')

## BART Data

#### Weekly Ridership by Month `1997 - 2018`

In [68]:
url = 'https://data.bart.gov/api/3/action/datastore_search?resource_id=6e653520-58cf-45c5-b40c-d37c8957ec77&limit=5'

bart_req = requests.get(url)
bart_req.status_code

# getting data 
request = bart_req.json()

In [70]:
bart_2018 = pd.DataFrame(request['result']['records'])
bart_2018.to_csv('../data/processed/bart_2018.csv')
bart_2018

,_id,FiscalYear,FiscalMonth,RIDERSHIP WEEKAVG,RIDERSHIP GOAL
0,1,1997,1,251524,240720
1,2,1997,2,256261,243974
2,3,1997,3,263602,253309
3,4,1997,4,264442,254655
4,5,1997,5,265244,259090


In [ ]:
# Weekly Ridership 2019 - 2022

In [ ]:
# # CA DATASETS TEST 
# # 
# # # https://www.cdtfa.ca.gov/dataportal/api/#/Alcohol_Consumption_Per_Capita/Alcohol_Consumption_Per_Capita_Get

# #url = 'https://www.cdtfa.ca.gov/dataportal/api/odata/Alcohol_Consumption_Per_Capita?%24filter=FiscalYearFrom%20gt%201987'
# #url = 'https://www.cdtfa.ca.gov/dataportal/api/odata/Alcohol_Consumption_Per_Capita?'

# url = 'https://www.cdtfa.ca.gov/dataportal/api/odata/Alcohol_Consumption_Per_Capita?%24filter=FiscalYearFrom%20gt%201935&%24select=FiscalYearFrom%2C%20PerCapitaDistilledSpiritsConsumptionGallons'

# # Make request
# req = requests.get(url)
# req.status_code

# sq = req.json()
# type(sq)

# sq.keys()

# mydf = pd.DataFrame(sq['value'])

# mydf.head()

# data = mydf

# # import seaborn as sns
# # sns.scatterplot(data = data, 
# #         x = 'FiscalYearFrom', 
# #         y = 'PerCapitaDistilledSpiritsConsumptionGallons')